In [73]:
import numpy as np
import re

# get list of experiments

In [91]:
gst_experiment_list = open("gst_measurements.txt", "r")
exp_list = list(gst_experiment_list)
gst_experiment_list.close()

In [117]:
gatelists = []
for expline in exp_list[1:]:
    gateline = expline.split('@')[0]
    linepowersplit = gateline.split('^')
    if(len(linepowersplit) == 2):
        power = int(linepowersplit[1][0])
        linepowersplit[1] = linepowersplit[1][1:]
#         print(power)
        pattern = "\((.*?)\)"
        substring = re.search(pattern, linepowersplit[0]).group(1)
#         print(substring)
        for j in range(power-1):
            linepowersplit[0] += substring
    gateline = "".join(linepowersplit)
    gateline = gateline.replace('(', "")
    gateline = gateline.replace(')', "")
    gatelist = gateline.split(':0')[:-1]
    gatelists.append(gatelist)
gatelists[0] = ["empty"]

In [122]:
gatelists[218]

['Gxpi2',
 'Gxpi2',
 'Gxpi2',
 'Gzpi2',
 'Gzpi2',
 'Gzpi2',
 'Gzpi2',
 'Gzpi2',
 'Gzpi2',
 'Gzpi2',
 'Gzpi2',
 'Gxpi2',
 'Gxpi2',
 'Gxpi2']

In [93]:
len(gatelists)

292

# generate master scripts

$Gx$ should include a z rotation to remove phase acculumated since last $Gx$ due to detuning of drive. $Gx_0(\pi/2)$ defined to be a pi/2 around x:

$$
Gx(\pi/2) = Gx_0(\pi/2)~ exp(-i \theta\sigma_z/2)
$$

For qubit splitting from field $\Delta_0$, and time $t$ since last $Gx$, $\theta = 2\pi \Delta_0 t$. And for splitting $\Delta_z$ under light shift for Z, the time of the Z part of the composite X is 

$$
t_z = \frac{\Delta_0}{\Delta_z} t
$$

$$
Gx(\pi/2) = Gx_0(\pi/2)~ Gz(2\pi \Delta_z t_z)
$$

In [119]:
1/(4*zpi2time)

454.5454545454545

In [138]:
f = open("qubit_empty_script.mScript", "r")
flist = list(f)

Gz = ["on: zrot\n","t += zpi2time\n","off: zrot\n", "t += 0.001\n"]

zpi2time = 0.00055
xpi2time = 0.00022
delta0 = 1.25519287
detlaz = 1/(4*zpi2time)

for j, gatelist in enumerate(gatelists):
    td = 0
    tz = 0
    f0 = open("gst_scripts/"+"".join(gatelist)+".mScript", "w")
    flist_new = []
    for i, line in enumerate(flist):
        if (line == "%STARTQUBIT\n"):
            flist_new = flist[:i+1].copy() 
            for gate in gatelist:
                if (gate == 'Gzpi2'):
                    td += zpi2time
                    flist_new += Gz
                if (gate == "Gxpi2"):
                    Gx = ["on: xrot\n","t += xpi2time\n","off: xrot\n","t += 0.001\n"]
                    tz = td*(delta0/detlaz)
                    nzpulses = int(np.round(tz/0.00005))
                    for k in range(nzpulses):
                        Gx = ["on: zrot\n","t += 0.00005\n","off: zrot\n","t += 0.001\n"] + Gx   
                    flist_new += Gx
                    td = xpi2time
                td += 0.001 
            flist_new += flist[i+1:]
            break
    for line in flist_new:
        f0.write(line)
    f0.close()
f.close()